# Задание 6 - 10 баллов

- Изучить [главу 3. Fine-tuning a pretrained model туториалов HuggingFace](https://huggingface.co/learn/nlp-course/chapter3/1)
- Выбрать модель архитектуры BERT/GPT в HuggingFace hub для решения задач Text Classification / Text Generation
- Выбрать набор данных для конкретной задачи из HuggingFace Datasets
- Дообучить выбранную модель - **5 баллов**
- Сравнить качество до и после дообучения с учетом метрик, специфичных для выбранной задачи - **2 балла**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **2 балла**

- Соблюден code style на уровне pep8 и [On writing clean Jupyter notebooks](https://ploomber.io/blog/clean-nbs/)  - **1 балл**

In [1]:
!pip install datasets transformers accelerate evaluate > None

In [2]:
import ast

from pprint import pprint

import numpy as np

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import EvalPrediction

import evaluate


SEED=566

# Данные

Буду использовать данные по ревью больниц, нас будет интересовать текстовый обзор (инпут) и general оценка (аутпут).


In [3]:
full_dataset = load_dataset("blinoff/medical_institutions_reviews")
full_dataset, full_dataset["train"][100]

(DatasetDict({
     train: Dataset({
         features: ['review_id', 'content', 'general', 'quality', 'service', 'equipment', 'food', 'location', 'Idx'],
         num_rows: 12036
     })
 }),
 {'review_id': '100',
  'content': 'современное оборудование, сеть клиник, нет очередей не обнаружено; Месяц назад надо было срочно посетить стоматолога. Позвонила к "своему", но ждать приема надо было две недели. Пришлось искать, кто примет быстрее. Через интернет нашла стоматологическую клинику "Мой зубной". Записали на прием через 4 дня. За день до назначенного времени позвонили из клиники, напомнили, что у меня запись на прием. В холле встретила администратор, завела на меня медицинскую карточку. Предложила бесплатные бахилы - удобный сервис, не надо думать о мелочах. В назначенное время за мной вышла лечащая врач. Внимательно меня выслушала, принесла договор на подпись, расписала для меня на отдельном листе план работы с подробным описанием цен за каждую услугу и преступила к лечению. Оборуд

Во первых замапим range Оценок 1-5 на 0-4

In [4]:
def label_to_int(example):
    example['labels'] = int(example['general']) - 1
    return example

full_dataset = full_dataset.map(label_to_int)

Так как есть только трэйн, вручную создадим трэйн-вал-тест

In [5]:
dataset = full_dataset['train'].train_test_split(test_size=0.4, seed=SEED)
dataset_va_te = dataset['test'].train_test_split(test_size=0.5, seed=SEED)
dataset = DatasetDict({'train': dataset['train'],
                       'validate': dataset_va_te['train'],
                       'test': dataset_va_te['test'],
                       })
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'content', 'general', 'quality', 'service', 'equipment', 'food', 'location', 'Idx', 'labels'],
        num_rows: 7221
    })
    validate: Dataset({
        features: ['review_id', 'content', 'general', 'quality', 'service', 'equipment', 'food', 'location', 'Idx', 'labels'],
        num_rows: 2407
    })
    test: Dataset({
        features: ['review_id', 'content', 'general', 'quality', 'service', 'equipment', 'food', 'location', 'Idx', 'labels'],
        num_rows: 2408
    })
})

In [6]:
X_col = 'content'
y_col = 'labels'

# Tokenization

Буду использовать базовый токенизатор и модель для русских тестов от DeepPavlov

In [7]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [8]:
%%time
def tokenize_function(examples):
    return tokenizer(examples[X_col], truncation=True, padding='max_length', max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7221 [00:00<?, ? examples/s]

Map:   0%|          | 0/2407 [00:00<?, ? examples/s]

Map:   0%|          | 0/2408 [00:00<?, ? examples/s]

CPU times: user 26.1 s, sys: 347 ms, total: 26.4 s
Wall time: 27.7 s


# Load base model

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Zero-shot predict

Воспользуемся интерфесом трейнера с отключенным `do_train` для оценки zero-shot.

In [10]:
metric_acc = evaluate.load("accuracy")
metric_f1 = evaluate.load("f1")

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': metric_acc.compute(predictions=predictions, references=labels)['accuracy'],
        'f1': metric_f1.compute(predictions=predictions, references=labels, average='weighted')['f1']
    }


def show_test_metrics():
  training_args = TrainingArguments(
      output_dir='./results_test',
      per_device_eval_batch_size=128,
      seed=SEED,
      do_train=False,  # Только оцениваем, не учим (!!!)
      do_eval=True,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      compute_metrics=compute_metrics,
      eval_dataset=tokenized_dataset['test'],  # Тестовый набор
  )

  results = trainer.evaluate()
  pprint(results)

show_test_metrics()

{'eval_accuracy': 0.08388704318936877,
 'eval_f1': 0.02620815866524628,
 'eval_loss': 1.612776756286621,
 'eval_runtime': 79.247,
 'eval_samples_per_second': 30.386,
 'eval_steps_per_second': 0.24}


Что и следовало ожидать -- результат такой себе, давайте дообучим.

# Fine-tuning

In [12]:
EPOCH = 10

training_args = TrainingArguments(
    output_dir='./results_train',
    num_train_epochs=EPOCH,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1,
    seed=SEED,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validate"],
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.639400,1.019086,0.611550,0.501540
2,0.685800,0.775166,0.678022,0.659060
3,1.868600,0.814983,0.668467,0.669218
4,0.232100,0.942347,0.684670,0.676464
5,1.278500,1.239033,0.645617,0.658562
6,0.005600,1.410862,0.657250,0.668746
7,0.617200,2.085940,0.613627,0.636439
8,0.003900,2.241995,0.663066,0.674142
9,0.000400,2.558499,0.660573,0.672145
10,0.000400,2.688239,0.651849,0.667214


TrainOutput(global_step=4520, training_loss=0.4035649423220399, metrics={'train_runtime': 7533.2207, 'train_samples_per_second': 9.586, 'train_steps_per_second': 0.6, 'total_flos': 1.899976106732544e+16, 'train_loss': 0.4035649423220399, 'epoch': 10.0})

In [13]:
show_test_metrics()

{'eval_accuracy': 0.6557308970099668,
 'eval_f1': 0.6426035707164227,
 'eval_loss': 0.819760262966156,
 'eval_runtime': 78.934,
 'eval_samples_per_second': 30.507,
 'eval_steps_per_second': 0.241}


# Выводы

Буквально на второй эпохе скор на валидэйте поднялся до (`acc~=0.6`, `acc~=0.6`), от которого сильно далеко не улучшился уже. Единственный мой вопрос: в какой-то момент видно по loss на тесте и трэйне, что модель пошла в переобучение. Как этого избежать, просто стопить, или есть более умные методы?